In [ ]:
!pip install nltk


In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from collections import defaultdict
from nltk.corpus import stopwords
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

nltk.download("stopwords")
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = pd.read_csv("captions.txt")

In [ ]:
images = df.iloc[:, 0]
captions = df.iloc[:, 1]
print(images.shape)
print(captions.shape)

(40455,)
(40455,)


In [ ]:
#### preprocess the captions #######
for i, caption in enumerate(captions):
    _caption = caption.lower()
    tokens = _caption.split()
    tokens = [word for word in tokens if not word in stop_words]
    # token length greater than one. removes dangling characters
    tokens = [word for word in tokens if len(word)>1]
    # remove tokens with numbers in them
    tokens = [word for word in tokens if word.isalpha()]
    captions[i] = " ".join(tokens)


print(captions[0:5])

0    child pink dress climbing set stairs entry way
1                        girl going wooden building
2             little girl climbing wooden playhouse
3             little girl climbing stairs playhouse
4         little girl pink dress going wooden cabin
Name: caption, dtype: object


In [ ]:
train_x, test_x, train_y, test_y = train_test_split(images, captions)
print(train_x[0:5])
print(train_y[0:5])

19590    3074842262_62b1b2168c.jpg
7529     2310108346_e82d209ccd.jpg
5269     2121357310_f8235311da.jpg
29123    3474265683_43b1033d94.jpg
331      1084104085_3b06223afe.jpg
Name: image, dtype: object
19590                                boy kicks pile leaves
7529                person sweater sits wood looking water
5269                       boy sliding wet plastic surface
29123    old wearing grey hat leather jacket surrounded...
331      woman blue shorts white shirt indoor rock clim...
Name: caption, dtype: object


In [ ]:
#### Create a vocabulary from the train captions in train_y #####

vocabulary = set()

for caption in train_y:
    tokens = caption.split(" ")
    for token in tokens:
        vocabulary.add(token)


In [ ]:
#### create a list of captions corresponding to a particular image ######

image_to_caption = defaultdict(list)

for image, caption in zip(images, captions):
    _image = image.replace(".jpg", "").strip()
    image_to_caption[_image].append(caption)




In [ ]:
model = InceptionV3(weights="imagenet")
model = Model(model.input, model.layers[-2].output)

In [ ]:
# Convert all the images to size 299x299 as expected by the
# encoded_images = []
# for image in images:
#   path = path.join("images", image)
#   img = image.load_img(path, target_size=(299, 299))
#   X = image.img_to_array(img)
#   X = np.expand_dims(x, axis=0)
#   X = preprocess_input(x)
#   X = np.reshape(x, x.shape[1])


image_path = path.join("images","1000268201_693b08cb0e.jpg")
img = image.load_img(image_path, target_size=(299, 299))
X = image.img_to_array(img)
X = np.expand_dims(X, axis=0)
X = preprocess_input(X)

X.shape
# X = np.reshape(X, X.shape[1])

print(X)